In [1]:
import tensorflow as tf
import numpy as np


In [ ]:
model_path = "results/julian/unet_5/model.tf"
model = tf.keras.models.load_model(model_path)
model.input.set_shape((1,256,256,3))
model.summary()

In [2]:
from custom import *
from create_model import generate_model
import tensorflow as tf

model = generate_model((256,256,3))
model.input.set_shape((1,256,256,3))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(1, 256, 256, 3)]   0           []                               
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 128, 128, 32  864         ['input_1[0][0]']                
                                )                                                                 
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 128, 128, 32  128         ['Conv1[0][0]']                  
                                )                                                                 
______________________________________________________________________________________________

In [4]:
def representative_data_gen():
  for _ in range(100):
    with open("plots/predictions/encoder_sC/img_0.npy", "rb") as f:
      img = np.load(f)
      img  = np.array(img, dtype=np.float32)
      yield [np.array([img])]



#converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# This sets the representative dataset for quantization
converter.representative_dataset = representative_data_gen
# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
converter.experimental_new_quantizer = True
# For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
converter.target_spec.supported_types = [tf.int8]
# These set the input and output tensors to uint8 (added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()


# Save the model.
with open('results/coral_chip/decoder/model.tflite', 'wb') as f:
  f.write(tflite_model)



INFO:tensorflow:Assets written to: /tmp/tmpxb9y10qm/assets


In [ ]:
from matplotlib import image
import numpy as np

with open("plots/predictions/encoder_sC/img_0.npy", "rb") as f:
    img = np.load(f)

image.imsave("test.png", img)